Retreive 20 records including the last 4 columns from the GCS's file generated from ex01.ipynb
Using "gemini-2.5-flash", ask it to summarize the reports.

- Pre-requisite : Generate API keys through [Google AI Studio](https://aistudio.google.com/app/apikey)

In [1]:
import json
import os

from dotenv import load_dotenv
from google import genai
from google.oauth2 import service_account
from google.cloud import storage

In [2]:
load_dotenv(dotenv_path='../.env')

True

In [3]:
genai_api_key = os.getenv("GCP_GENAI_API_KEY")
service_account_key = os.getenv("GCP_SERVICE_ACCOUNT_KEY")
project_id = os.getenv("GCP_PROJECT_ID")
bucket_name = os.getenv("GCP_BUCKET_NAME")
file_name = f"sf_police_report/2025-09-01.json"

In [4]:
def retrieve_data_from_gcs(service_account_key: str,
                           project_id: str,
                           bucket_name: str,
                           file_name: str,
                           key_list: list
                           ) -> list:
    credentials = service_account.Credentials.from_service_account_file(service_account_key)
    client = storage.Client(project=project_id,
                            credentials=credentials)
    bucket = client.bucket(bucket_name)
    file = bucket.blob(file_name)
    content = json.loads(file.download_as_string())

    output = []
    for data in content:
        row = []
        
        for key in key_list:
            row.append(data.get(key, None))
        output.append(row)
    return output

In [5]:
key_list = ['incident_datetime', 'report_datetime', 'incident_code',
            'incident_category', 'incident_description', 'latitude',
            'longitude', 'police_district']
data = retrieve_data_from_gcs(service_account_key, 
                              project_id,
                              bucket_name,
                              file_name,
                              key_list)

In [6]:
filtered_data = [row[-4:] for row in data][:20]

In [7]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [8]:
model_name = "gemini-2.5-flash"

In [9]:
prompt_content = f"There has been a police report\
on the following list of [description, lon, lat, district] : {filtered_data} recently.\
Summarize the reports"

In [10]:
response = client.models.generate_content(
    model=model_name,
    contents=prompt_content
)

In [11]:
response.text

'A total of **20 police reports** were filed, covering a variety of incidents across multiple districts.\n\nHere\'s a summary of the key findings:\n\n1.  **Vehicle-Related Crimes (8 incidents):**\n    *   **Theft from Vehicles** is a significant category, with **4 reports** of "Theft, From Locked Vehicle, >$950" (Northern, Southern x2, Park) and **1 report** of "Theft, From Unlocked Vehicle, >$950" (Southern).\n    *   **Vehicle Theft/Recovery:** One "Vehicle, Stolen, Other Vehicle" (Park) and one "Vehicle, Recovered, Auto" (Out of SF) were reported.\n    *   **Other:** One "Arson of Vehicle" (Tenderloin) and one "Evading a Police Officer Recklessly" (Southern) also occurred.\n\n2.  **Assault, Threats, and Disorderly Conduct (6 incidents):**\n    *   Reports include "Battery" (Park), "False Personation" (Park), "Malicious Mischief, Vandalism to Property" (Taraval), "Resisting, Delaying, or Obstructing Peace Officer Duties" (Mission), "Terrorist Threats" (Central), and "Phone Calls, Har